In [2]:
#This will run the createQualificationType in the commandline and will create a qualification type in MTurk
#You need to find these text files in your old computer
#You should make these text files more easily editable through this python script as well (read in files, edit, output)
import os
import subprocess
os.chdir('C:\\Users\\bodil\\aws-mturk-clt-1.3.3\\bin')
args = ["createQualificationType","-properties",'properties.txt','-answer','answer_key.txt','-question','test.txt','-noretry']
args = subprocess.list2cmdline(args)
os.system(args)

0

In [3]:
for resp in qualtrics_responses:
    ###change this question based on the new qualtrics survey###
    if qualtrics_responses[resp]['Q31'] == 1:
        qualtrics_codes.append(qualtrics_responses[resp]['survey_code'])
    else:
        pass

In [4]:
#This code will get all submitted qualtrics survey codes
#Then it will get all submitted MTurk codes
#Then it compares them and approves the ones that match and rejects the ones that do not
#It also makes a list of workers and changes their qualification number so that they cannot take the survey again in the future
###########################################################
#change this value to the actual qualification type id
qualification_type_id = 'INSERT ID HERE'
###########################################################

import math
import requests
import boto.mturk.connection
connection = boto.mturk.connection.MTurkConnection(aws_access_key_id='INSERT ID HERE', 
                                      aws_secret_access_key='INSERT KEY HERE', 
                                      is_secure=True, port=None, 
                                      proxy=None, proxy_port=None, 
                                      proxy_user=None, proxy_pass=None, 
                                      host='mechanicalturk.amazonaws.com', debug=0, 
                                      https_connection_factory=None, 
                                      security_token=None, profile_name=None)

url = "https://survey.qualtrics.com/WRAPI/ControlPanel/api.php?Request=getLegacyResponseData&Version=2.0&User=[INSERT USER]&Token=[INSERT TOKEN]"
qualtrics_codes = []
qualtrics_responses = requests.get(url).json()
for resp in qualtrics_responses:
    ###change this question based on the new qualtrics survey###
    if qualtrics_responses[resp]['Q31'] == 1:
        qualtrics_codes.append(qualtrics_responses[resp]['survey_code'])
    else:
        pass

url = "https://survey.qualtrics.com/WRAPI/ControlPanel/api.php?Request=getLegacyResponseData&ResponsesInProgress=1&Version=2.0&User=[INSERT USER]&Token=[INSERT TOKEN]&Format=JSON&SurveyID={INSERT ID}"
qualtrics_responses = requests.get(url).json()
for resp in qualtrics_responses:
    ###change this question based on the new qualtrics survey###
    if qualtrics_responses[resp]['Q31'] == 1:
        qualtrics_codes.append(qualtrics_responses[resp]['survey_code'])
    else:
        pass

workers = []
approved = []
rejected = []
all_hits = [hit for hit in connection.get_all_hits()]
codes_entered_to_mturk = []
#testcode hitcode = '3CRWSLD91K4646XDRA2O9WPNTE4OM0'
for hit in all_hits:
    assignments = connection.get_assignments(hit.HITId,status='Rejected',page_size=100,page_number=1)
    number_of_pages = int(math.ceil(float(assignments.TotalNumResults)/100))
    for number in range(2,number_of_pages+1):
        assignments += connection.get_assignments(hit.HITId,status='Submitted',page_size=100,page_number=number)
    for assignment in assignments:
        workers.append(assignment.WorkerId)
        # don't ask me why this is a 2D list
        question_form_answers = assignment.answers[0]
        for question_form_answer in question_form_answers:
            # "user-input" is the field I created and the only one I care about
            if question_form_answer.qid == "surveycode":
                user_response = question_form_answer.fields[0]
                user_response = user_response.replace(' ','')
                codes_entered_to_mturk.append(user_response)
        if user_response in qualtrics_codes:
            approved.append((assignment.AssignmentId,assignment.WorkerId,user_response))
#             connection.approve_assignment(assignment.AssignmentId)
        else:
#             connection.reject_assignment(assignment.AssignmentId, feedback="I'm sorry, your code was not found in our qualtrics database of generated codes. Please contact the email address associated with the requesters MTurk account if you feel you have been rejected in error.")
            rejected.append((assignment.AssignmentId,assignment.WorkerId,user_response))
# workers = list(set(workers))
# for worker in workers:
#     value = -1
#     connection.update_qualification_score(qualification_type_id, worker, value)

In [3]:
user_response = user_response.replace(' ','')

In [3]:
#This script will approve a rejection due to technical difficulties (maybe we have their responses, just not on Q18...?)
import math
import requests
import boto.mturk.connection

feedback = '''
I am sorry for the rejection. There are a number of reasons this may have occurred:

1. The most common reason is if you forgot to hit submit in qualtrics after you completed the survey.
2. Another reason is the code you entered was incorrect (space before or after the code or incorrect code)

Thank you for contacting me. I have reversed the rejection. I hope you will continue to do work for me in the future.
'''
feedback = '''
I am sorry for the rejection. You were rejected because when you put the Qualtrics code into MTURK, you accidentally did
not click finish on the survey. This meant your response was still in progress, and so your HIT was automatically rejected. 
We are sorry that we accidentally rejected your work. We appreciate the time you spent working on this hit.

Thank you for contacting me. I have reversed the rejection. I hope you will continue to do work for me in the future.

'''
feedback = '''
I am sorry for the rejection. Your HIT was rejected because the code you entered had a space before the code, so it did 
not match with any codes in our database.

Thank you for contacting me. The code was correct (other than the space).

I have reversed the rejection. I hope you will continue to do work for me in the future.
'''
connection = boto.mturk.connection.MTurkConnection(aws_access_key_id='INSERT ID HERE', 
                                      aws_secret_access_key='INSERT KEY HERE', 
                                      is_secure=True, port=None, 
                                      proxy=None, proxy_port=None, 
                                      proxy_user=None, proxy_pass=None, 
                                      host='mechanicalturk.amazonaws.com', debug=0, 
                                      https_connection_factory=None, 
                                      security_token=None, profile_name=None)

In [4]:
assignment_ids_for_approval = ['INSERT ID HERE']

In [1]:
#Go through and approve the rejected assignments
for assignment_id in assignment_ids_for_approval:
    try:
        connection.approve_rejected_assignment(assignment_id, feedback=feedback)
        print assignment_id
    except:
        print 'failure'
    

In [2]:
import requests
url = "https://survey.qualtrics.com/WRAPI/ControlPanel/api.php?Request=getLegacyResponseData&Version=2.0&User=[INSERT USER]&Token=[INSERT TOKEN]&Format=JSON&SurveyID=[INSERT ID]"
qualtrics_codes = []
qualtrics_responses = requests.get(url).json()